In [43]:
#Importing read-sm-files.py
import ReadSMFIles
from Resource import Resource
from Task import Task
from Schedule import Schedule
import random
from typing import List, Tuple

#reading j30.sm/j301_1.sm file
sm_file = ReadSMFIles.SMFileParser.parse_sm_file("j30.sm/j301_1.sm")

print(sm_file)


(   renewable  nonrenewable  doubly_constrained
0          4             0                   0, {'pronr': 1, 'jobs': 30, 'rel_date': 0, 'duedate': 38, 'tardcost': 26, 'MPM_Time': 38},     jobnr  modes  successors_count    successors
0       1      1                 3     [2, 3, 4]
1       2      1                 3   [6, 11, 15]
2       3      1                 3    [7, 8, 13]
3       4      1                 3    [5, 9, 10]
4       5      1                 1          [20]
5       6      1                 1          [30]
6       7      1                 1          [27]
7       8      1                 3  [12, 19, 27]
8       9      1                 1          [14]
9      10      1                 2      [16, 25]
10     11      1                 2      [20, 26]
11     12      1                 1          [14]
12     13      1                 2      [17, 18]
13     14      1                 1          [17]
14     15      1                 1          [25]
15     16      1                

In [44]:
#Creating resources
r1 = int(sm_file[4].R1[0])
r2 = int(sm_file[4].R2[0])
r3 = int(sm_file[4].R3[0])
r4 = int(sm_file[4].R4[0])

R1 = Resource('R1', r1)
R2 = Resource('R2', r2)
R3 = Resource('R3', r3)
R4 = Resource('R4', r4)

resources = [R1, R2, R3, R4]

print([resource.name for resource in resources], [resource.per_period_availability for resource in resources])


['R1', 'R2', 'R3', 'R4'] [12, 13, 4, 12]


In [45]:
#Creating jobs
jobs_enumerate = sm_file[3].jobnr
jobs_duration = sm_file[3].duration
jobs_resources = sm_file[3].resources
jobs_successors = sm_file[2].successors

jobs = [None for _ in jobs_enumerate]


for i in jobs_enumerate:
    jobs[i - 1] = Task(str(i), jobs_duration[i - 1])

for i in range(len(resources)):
    for j in range(len(jobs)):
        jobs[j].add_renewable_resource(resources[i], jobs_resources[j][i])

for i in range(len(jobs)):
    successors = jobs_successors[i]
    for j in successors:
        jobs[i].add_sucessor(jobs[j - 1])
    
for job in jobs:
    print(job.name, [sucessor.name for sucessor in job.predecessors])

    

    



1 []
2 ['1']
3 ['1']
4 ['1']
5 ['4']
6 ['2']
7 ['3']
8 ['3']
9 ['4']
10 ['4']
11 ['2']
12 ['8']
13 ['3']
14 ['9', '12']
15 ['2']
16 ['10']
17 ['13', '14']
18 ['13']
19 ['8']
20 ['5', '11', '18']
21 ['16']
22 ['16', '17', '18']
23 ['20', '22']
24 ['19', '23']
25 ['10', '15', '20']
26 ['11']
27 ['7', '8']
28 ['21', '27']
29 ['19']
30 ['6', '24', '25']
31 ['26', '28']
32 ['29', '30', '31']



Genetic Algorithm



In [46]:
#Genetic Algorithm

#Creating random population
def create_population(population_size, jobs, resources: List[Resource]):
    population = []
    for i in range(population_size):
        schedule = Schedule()
        schedule.add_renewable_resources(resources)
        copy = jobs.copy()
        random.shuffle(copy)
        schedule.add_tasks(copy)
        population.append(schedule)
    
    return population

def create_population_with_valid_individuals(population_size: int, jobs: List[Task], resources: List[Resource]) -> List[Schedule]:
    population = []
    for _ in range(population_size):
        schedule = Schedule()
        schedule.add_renewable_resources(resources)
        _jobs = jobs.copy()
        while len(_jobs) > 0:
            job = random.choice(_jobs)
            schedule.add_task(job)
            if schedule.is_valid_precedence_relations_constraint():
                _jobs.remove(job)
            else:
                schedule.remove_task(job)
        population.append(schedule)
    return population


population = create_population(200, jobs, resources)

for individual in population:
    print([task.name for task in individual.tasks], individual.makespan())


['Dummy Source', '27', '11', '31', '24', '10', '29', '6', '1', '9', '3', '14', '13', '8', '4', '28', '26', '5', '15', '16', '17', '2', '19', '12', '7', '30', '25', '18', '32', '23', '21', '20', '22', 'Dummy Sink'] 50
['Dummy Source', '3', '13', '30', '6', '5', '25', '28', '11', '21', '31', '22', '14', '23', '2', '18', '29', '19', '16', '12', '1', '8', '4', '10', '7', '26', '27', '17', '32', '15', '24', '20', '9', 'Dummy Sink'] 57
['Dummy Source', '30', '14', '10', '22', '31', '26', '18', '9', '29', '11', '28', '8', '25', '12', '20', '6', '2', '17', '3', '21', '7', '24', '4', '13', '19', '27', '15', '23', '16', '32', '1', '5', 'Dummy Sink'] 46
['Dummy Source', '19', '31', '25', '23', '5', '24', '9', '10', '29', '26', '22', '28', '12', '15', '18', '2', '20', '4', '6', '8', '11', '3', '14', '32', '1', '21', '13', '7', '30', '27', '16', '17', 'Dummy Sink'] 53
['Dummy Source', '7', '15', '3', '14', '6', '31', '16', '4', '26', '1', '21', '8', '19', '27', '18', '23', '13', '22', '25', '12', '

Crossover

In [51]:
for individual in population:
    print(individual.is_valid_precedence_relations_constraint())

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [48]:

class OnePointCrossoverOperator:
    def __init__(self, crossover_rate: float = 0.8):
        self.crossover_rate = crossover_rate

    def __call__(
            self, parent1: Schedule, parent2: Schedule
    ) -> Tuple[Schedule, Schedule]:
        if random.random() > self.crossover_rate:
            return parent1, parent2

        n = len(parent1.tasks)
        crossover_point = random.randint(1, n - 3)

        child1 = Schedule()
        child2 = Schedule()

        parent1_tasks = parent1.tasks.copy()[1:-1]
        parent2_tasks = parent2.tasks.copy()[1:-1]

        child1.add_tasks(parent1_tasks[:crossover_point])
        for task in parent2.tasks:
            if task not in child1.tasks and task.name != 'Dummy Source' and task.name != 'Dummy Sink':
                child1.add_task(task)
        
        child2.add_tasks(parent2_tasks[:crossover_point])
        for task in parent1.tasks:
            if task not in child2.tasks and task.name != 'Dummy Source' and task.name != 'Dummy Sink':
                child2.add_task(task)

        return child1, child2


crossover = OnePointCrossoverOperator()

offspring = []
for i in range(0, len(population)):
    parent1 = random.choice(population)
    parent2 = random.choice(population)
    child1, child2 = crossover(parent1, parent2)
    offspring.append(child1)


#PRINTING TIME INTERVALS
# for task in child1.tasks:
#     print(task.earliest_start, task.earliest_finish, task.duration)




Parent Selection

In [49]:
class TournamentSelector:
    def __init__(self, tournament_size: int = 2):
        self.tournament_size = tournament_size

    def __call__(self, population: List[List[Task]]) -> List[Task]:
        selected = []
        for _ in range(len(population)):
            tournament = random.sample(population, self.tournament_size)
            winner = min(tournament, key=lambda x: x.makespan())
            selected.append(winner)
        return selected


selector = TournamentSelector(tournament_size=10)

selected = selector(population)

for i in range(len(selected)):
    print(selected[i].is_valid_precedence_relations_constraint(), [task.name for task in selected[i].tasks], selected[i].makespan())

False ['Dummy Source', '5', '4', '25', '23', '14', '13', '2', '22', '9', '3', '30', '21', '12', '20', '15', '26', '28', '7', '6', '1', '24', '10', '16', '27', '19', '17', '11', '18', '31', '29', '8', '32', 'Dummy Sink'] 51
False ['Dummy Source', '20', '18', '7', '15', '2', '13', '16', '27', '21', '9', '26', '3', '23', '24', '25', '12', '11', '14', '4', '29', '30', '19', '17', '6', '22', '8', '1', '10', '5', '28', '32', '31', 'Dummy Sink'] 52
False ['Dummy Source', '2', '1', '21', '20', '7', '19', '26', '6', '12', '24', '17', '31', '11', '32', '14', '9', '22', '16', '23', '5', '4', '25', '15', '18', '10', '29', '3', '28', '8', '13', '27', '30', 'Dummy Sink'] 55
False ['Dummy Source', '30', '14', '10', '22', '31', '26', '18', '9', '29', '11', '28', '8', '25', '12', '20', '6', '2', '17', '3', '21', '7', '24', '4', '13', '19', '27', '15', '23', '16', '32', '1', '5', 'Dummy Sink'] 58
False ['Dummy Source', '19', '31', '25', '23', '5', '24', '9', '10', '29', '26', '22', '28', '12', '15', '18